# CREATION DE LA BDD

---------------------------------------------
----------------------------------------

### creation de la bdd collection dans postgresql

In [1]:
# import des librairies
import psycopg2  # librairie pour utiliser postgresql
import os # module qui nous permet d'utiliser les variables d'environnements

In [ ]:
"""
Connexion dans un premier temps à postgres
pour initialiser notre database collection
"""



utilisateur = "postgres"

mot_passe = os.environ.get('pg_psw')

base_de_donnees = "postgres"



try:
    
    # création de l'objet conn

    conn = psycopg2.connect(dbname= base_de_donnees, user=utilisateur, password=mot_passe, host="localhost", port= 5432)

    conn.set_session(autocommit=True)   # on force

    # initialisation du cursor

    cursor = conn.cursor()

    # cursor.execute permet d'exécuter une commande 'sql'

    cursor.execute("CREATE DATABASE collection")

    print("La DataBase à été créée avec succès !")

        
except psycopg2.Error as e:
    
    print("Problème lors de la connection à la base de donnée")

    print(e)

   
    
finally:
        
    # ferme le curseur et devient inutilisable

    cursor.close

    # ferme la connexion à database

    conn.close

--------------------------------

fonctions de gestion base de données

In [ ]:
# Cellule pour nos fonctions

# Fonction qui va nous permettre de créer une Base de données.
def ouvrir_connection(base_de_donnee, utilisateur, mot_passe, host='localhost', port=5432):
    
    try:
        # création de l'objet conn
        conn = psycopg2.connect(dbname= base_de_donnee, user=utilisateur, password = mot_passe, host="localhost", port= 5432)

        # on force la sauvegarde
        conn.set_session(autocommit=True)   
        
        # initialisation du cursor
        cursor = conn.cursor()
        
        # exécution de la requête sql de création de la base de donnée
        #cursor.execute("CREATE DATABASE IF NOT EXISTS movie")

    # gestion des erreurs    
    except psycopg2.Error as e:
        print("Problème lors de la connection à la base de donnée")
        print(e)
        return None
    # dans tous les cas on va clôturer la session
    finally:
        cursor.close
        conn.close
    return conn
   
    # fonction pour supprimer une table
def supprimer_table(conn, sql_suppression_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_suppression_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la suppression de la table")
        print(e)
        return
    cursor.close()
    print("La table a été supprimée avec succès")

# fonction pour créer une table    
def creer_table(conn, sql_creer_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_creer_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la création de la table")
        print(e)
        return
    cursor.close()
    print("La table a été crée avec succès")
    
def lire_table(conn, sql_lire_donnee):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_lire_donnee)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la lecture des données")
        print(e)
        return None
    
    print("Les données ont été lues avec succès")
    data = []
    for row in cursor:
        data.append(row)

    cursor.close()
    
    return data   

-----------------------------------

requêtes sql

In [ ]:
# cellule qui content différentes requête sql

sql_creer_table1 = """CREATE TABLE IF NOT EXISTS Origine(
   Code_Origine SERIAL,
   Nom VARCHAR(100) NOT NULL,
   PRIMARY KEY(Code_Origine)
);
"""
sql_creer_table2 = """CREATE TABLE IF NOT EXISTS Producteur(
   Code_producteur SERIAL,
   Nom VARCHAR(100),
   Code_Origine INT NOT NULL,
   PRIMARY KEY(Code_producteur),
   FOREIGN KEY(Code_Origine) REFERENCES Origine(Code_Origine)
);
"""
sql_creer_table3 = """CREATE TABLE IF NOT EXISTS Produit(
   Code_produit SERIAL,
   Nom VARCHAR(250),
   Volume INT,
   Degre DECIMAL(15,2),
   Code_producteur INT,
   PRIMARY KEY(Code_produit),
   FOREIGN KEY(Code_producteur) REFERENCES Producteur(Code_producteur)
);
"""

sql_lire_donnee = """
    SELECT *
    FROM Produit
    LIMIT 10;
"""

In [ ]:
# initialisation des variables pour me connecter à ma bdd postgres

base_de_donnee = "collection"

utilisateur = "postgres"

mot_passe = os.environ.get('pg_psw')


In [ ]:
# initialisation de ma connection
conn = ouvrir_connection(base_de_donnee, utilisateur, mot_passe)

In [ ]:
# création table origine
creer_table(conn, sql_creer_table1)

In [ ]:
# création table porducteur
creer_table(conn, sql_creer_table2)

In [ ]:
# création table produit
creer_table(conn, sql_creer_table3)

------------------------------------

### nettoyage des données de l'inventaire perso

In [1]:
# on tuilise la librairie pandas pour nettoyer mes données

import pandas as pd

excel_file = 'INVENTAIRE_B.xlsx'
inventaire = pd.read_excel(excel_file, skiprows=2)
del inventaire['CADEAU']
inventaire.head()

,NOM,VOLUME,DEGRE,NOMBRE,PRIX,MARQUE
0,C12 70,70,50.0,5,8.5,HSE
1,C16 70,70,50.0,1,11.0,HSE
2,C16 100,100,50.0,1,14.0,HSE
3,PARCELLAIRE 70,70,55.0,2,13.0,HSE
4,PARCELLAIRE 100,100,55.0,1,15.0,HSE


In [17]:
inventaire

,NOM,VOLUME,DEGRE,NOMBRE,PRIX,MARQUE
0,C12 70,70,50.0,5,8.50,HSE
1,C16 70,70,50.0,1,11.00,HSE
2,C16 100,100,50.0,1,14.00,HSE
3,PARCELLAIRE 70,70,55.0,2,13.00,HSE
4,PARCELLAIRE 100,100,55.0,1,15.00,HSE
...,...,...,...,...,...,...
85,MONTEBELLO 8 ANS,70,42.0,1,28.00,montebello
86,kapoeira cachaça,70,38.0,1,25.00,kapoeira
87,Bombay Gin,70,40.0,1,15.00,bombay
88,Vodka Standart,100,40.0,1,24.00,standart


In [2]:
excel_file = 'INVENTAIRE_A.xlsx'
inventaire = pd.read_excel(excel_file, skiprows=2)
inventaire.drop(99,0,inplace=True)
inventaire.drop(0,0,inplace=True)
del inventaire['Unnamed: 5']
inventaire.columns = ['NOM', 'VOLUME', 'DEGRE', 'NOMBRE', 'PRIX', 'MARQUE']
inventaire.dropna(axis = 0, how = 'any')
inventaire

,NOM,VOLUME,DEGRE,NOMBRE,PRIX,MARQUE
1,C12 70,70.0,50.0,5.0,8.5,HSE
2,C16 70,70.0,50.0,1.0,11.0,HSE
3,C16 100,100.0,50.0,1.0,14.0,HSE
4,PARCELLAIRE 70,70.0,55.0,2.0,13.0,HSE
5,PARCELLAIRE 100,100.0,55.0,1.0,15.0,HSE
...,...,...,...,...,...,...
94,GUADELOUPE,NaN,NaN,NaN,NaN,NaN
95,BOLOGNE BLACK CANE N°19658,70.0,50.0,1.0,21.0,Bologne
96,BOLOGNE BLACK CANE N°19742,70.0,50.0,1.0,21.0,Bologne
97,BARBANCOURT VSOP 4 ans,70.0,40.0,1.0,21.0,Barbancourt


In [19]:
print(inventaire['MARQUE'].isnull())

1      False
2      False
3      False
4      False
5      False
       ...  
98     False
99      True
100    False
101    False
102    False
Name: MARQUE, Length: 102, dtype: bool


In [20]:
inventaire.isnull().sum().sum()

65

In [21]:
index_with_nan = inventaire.index[inventaire.isnull().any(axis=1)]

print(index_with_nan)

Int64Index([39, 48, 58, 64, 68, 72, 76, 79, 85, 88, 91, 94, 99], dtype='int64')


In [22]:
print(inventaire.isnull().sum(axis=1))

1      0
2      0
3      0
4      0
5      0
      ..
98     0
99     5
100    0
101    0
102    0
Length: 102, dtype: int64


In [23]:
inventaire.drop(index_with_nan,0, inplace=True)
print(inventaire)

                        NOM  VOLUME  DEGRE  NOMBRE  PRIX       MARQUE
1                    C12 70    70.0   50.0     5.0   8.5          HSE
2                    C16 70    70.0   50.0     1.0  11.0          HSE
3                   C16 100   100.0   50.0     1.0  14.0          HSE
4            PARCELLAIRE 70    70.0   55.0     2.0  13.0          HSE
5           PARCELLAIRE 100   100.0   55.0     1.0  15.0          HSE
..                      ...     ...    ...     ...   ...          ...
97   BARBANCOURT VSOP 4 ans    70.0   40.0     1.0  21.0  barbancourt
98         MONTEBELLO 8 ANS    70.0   42.0     1.0  28.0   montebello
100       kapoeira  cachaça    70.0   38.0     1.0  25.0     kapoeira
101              Bombay Gin    70.0   40.0     1.0  15.0       bombay
102          Vodka Standart   100.0   40.0     1.0  24.0     standart

[89 rows x 6 columns]


-----------------------------------

insertion des données dans la bdd postgres

In [24]:
from sqlalchemy import create_engine
import os

user = 'postgres'
password = os.environ.get('pg_psw')
host = 'localhost'
port = '5432'
dbname = 'collection'  

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{dbname}")


In [25]:
mesure_produit = inventaire[['NOM', 'VOLUME', 'DEGRE']]

In [28]:
mesure_produit.to_sql('produit', engine, if_exists='replace', index=False)

In [29]:
inventaire.dtypes

NOM        object
VOLUME    float64
DEGRE     float64
NOMBRE    float64
PRIX      float64
MARQUE     object
dtype: object

In [30]:
inventaire.tail(15)

,NOM,VOLUME,DEGRE,NOMBRE,PRIX,MARQUE
83,RENNAISSANCE PORSCHE n°67,70.0,52.0,1.0,55.0,A1713
84,AGRONOME N°0704,70.0,50.0,1.0,30.0,A1714
86,XO,70.0,43.0,1.0,24.0,SAINT JAMES
87,12 ANS,70.0,43.0,1.0,40.0,SAINT JAMES
89,XO,70.0,40.0,1.0,35.0,LAMAUNY
90,VSOP,70.0,40.0,1.0,22.0,LAMAUNY
92,"2005, 12 ans, octobre 2017",70.0,59.0,2.0,35.0,FOURSQUARE
93,DOMINUS 10 years-01/2018,70.0,56.0,1.0,25.0,FOURSQUARE
95,BOLOGNE BLACK CANE N°19658,70.0,50.0,1.0,21.0,bologne
96,BOLOGNE BLACK CANE N°19742,70.0,50.0,1.0,21.0,bologne


In [31]:
inventaire.head(9)

,NOM,VOLUME,DEGRE,NOMBRE,PRIX,MARQUE
1,C12 70,70.0,50.0,5.0,8.5,HSE
2,C16 70,70.0,50.0,1.0,11.0,HSE
3,C16 100,100.0,50.0,1.0,14.0,HSE
4,PARCELLAIRE 70,70.0,55.0,2.0,13.0,HSE
5,PARCELLAIRE 100,100.0,55.0,1.0,15.0,HSE
6,PARCELLAIRE 150L,150.0,55.0,1.0,40.0,HSE
7,RAGTIME ESB,70.0,40.0,1.0,11.0,HSE
8,Coffret XO ENO,70.0,43.0,1.0,40.0,HSE
9,"Small Cask 2007-22/10/2015, n°848",50.0,46.0,1.0,26.0,HSE


In [32]:
inventaire.info() # résumé technique de la dataframe

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89 entries, 1 to 102
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   NOM     89 non-null     object 
 1   VOLUME  89 non-null     float64
 2   DEGRE   89 non-null     float64
 3   NOMBRE  89 non-null     float64
 4   PRIX    89 non-null     float64
 5   MARQUE  89 non-null     object 
dtypes: float64(4), object(2)
memory usage: 4.9+ KB


---------------------

### Fichier csv rum_data, manipulation des données

- import et lecture des données

In [1]:
# import des librairies
import psycopg2  # librairie pour utiliser postgresql
import os # module qui nous permet d'utiliser les variables d'environnements
import pandas as pd

csv_file = 'rum_data.csv'
list_rhum = pd.read_csv(csv_file)

list_rhum.head()

,name,company,country,price,ratings,score,type,rum_url,img_url,br_score
0,10000 Drops Silver,Unknown,United States,0,0,0.0,Light,/brands/10972-10000-drops-silver,https://d1jtwiy8m5zi8a.cloudfront.net/assets/d...,4.775363
1,10000 Drops Spiced,Unknown,United States,0,1,4.0,Spiced,/brands/7354-10000-drops-spiced,https://d19vk5i0q1xlsm.cloudfront.net/uploads/...,4.711955
2,1000 Hills Gold,Unknown,Rwanda,0,1,4.0,Light,/brands/5285-1000-hills-gold,https://d19vk5i0q1xlsm.cloudfront.net/uploads/...,4.711955
3,100 Fuegos Buckeye 2-Year,Unknown,Ecuador,0,0,0.0,Gold,/brands/9037-100-fuegos-buckeye-2-year,https://d1jtwiy8m5zi8a.cloudfront.net/assets/d...,4.775363
4,100 Fuegos 8-Year,Unknown,Ecuador,0,0,0.0,Aged,/brands/1834-100-fuegos-8-year,https://d1jtwiy8m5zi8a.cloudfront.net/assets/d...,4.775363


In [2]:
list_rhum.tail()

,name,company,country,price,ratings,score,type,rum_url,img_url,br_score
6883,Yolo Gold 10-Year,Yolo,Panama,0,4,8.0,Gold,/brands/1588-yolo-gold-10-year,https://d19vk5i0q1xlsm.cloudfront.net/uploads/...,5.622387
6884,Yolo Silver,Yolo,Panama,0,1,7.0,Light,/brands/1587-yolo-silver,https://d19vk5i0q1xlsm.cloudfront.net/uploads/...,4.957292
6885,Ypioca 150 Cachaca,Ypioca,Brazil,0,1,7.0,Cachaca,/brands/7212-ypioca-150-cachaca,https://d19vk5i0q1xlsm.cloudfront.net/uploads/...,4.957292
6886,Ypioca Cachaca 160,Ypioca,Brazil,0,2,6.5,Agricole,/brands/6194-ypioca-cachaca-160,https://d19vk5i0q1xlsm.cloudfront.net/uploads/...,5.036117
6887,Ypioca Chrystal Cachaca,Ypioca,Brazil,0,6,6.5,Cachaca,/brands/1276-ypioca-chrystal-cachaca,https://d19vk5i0q1xlsm.cloudfront.net/uploads/...,5.376000


In [3]:
list_rhum.isnull()

,name,company,country,price,ratings,score,type,rum_url,img_url,br_score
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
6883,False,False,False,False,False,False,False,False,False,False
6884,False,False,False,False,False,False,False,False,False,False
6885,False,False,False,False,False,False,False,False,False,False
6886,False,False,False,False,False,False,False,False,False,False


In [4]:
list_rhum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6888 entries, 0 to 6887
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      6888 non-null   object 
 1   company   6888 non-null   object 
 2   country   6888 non-null   object 
 3   price     6888 non-null   int64  
 4   ratings   6888 non-null   int64  
 5   score     6888 non-null   float64
 6   type      6888 non-null   object 
 7   rum_url   6888 non-null   object 
 8   img_url   6888 non-null   object 
 9   br_score  6888 non-null   float64
dtypes: float64(2), int64(2), object(6)
memory usage: 538.2+ KB


In [39]:
list_rhum.columns = ['NOM', 'PRODUCTEUR', 'ORIGINE', 'PRIX', 'NOTES', 'SCORE', 'TYPE', 'RUM_URL', 'IMG_URL', 'BR_SCORE']

- nettoyage des données

In [40]:
countries = list_rhum['ORIGINE']

In [41]:
countries 

0       United States
1       United States
2              Rwanda
3             Ecuador
4             Ecuador
            ...      
6883           Panama
6884           Panama
6885           Brazil
6886           Brazil
6887           Brazil
Name: ORIGINE, Length: 6888, dtype: object

In [42]:
countries = countries.drop_duplicates()

In [43]:
countries

0             United States
2                    Rwanda
3                   Ecuador
6       Trinidad and Tobago
9                    Panama
               ...         
6276          Liechtenstein
6415    British West Indies
6572             Cape Verde
6759                 Brasil
6762                Ireland
Name: ORIGINE, Length: 111, dtype: object

In [44]:
brand = list_rhum['PRODUCTEUR'].drop_duplicates()

In [45]:
brand

0          Unknown
6          10 Cane
9             1423
12            1492
30            1767
           ...    
6870     Xtabentun
6875        Yacuro
6880    Yahara Bay
6883          Yolo
6885        Ypioca
Name: PRODUCTEUR, Length: 780, dtype: object

In [46]:
product = list_rhum['NOM']

- insertion des données

In [47]:
from sqlalchemy import create_engine
import os

user = 'postgres'
password = os.environ.get('pg_psw')
host = 'localhost'
port = '5432'
dbname = 'collection'  

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{dbname}")

#inventaire.to_sql('rhum', engine, if_exists='replace', index=False)


In [48]:
brand.to_sql('producteur', engine, if_exists='replace', index=False)

In [49]:
countries.to_sql('origine', engine, if_exists='replace', index=False)

In [50]:
product.to_sql('produit', engine, if_exists='append', index=False)

-------------------

### ajout des id auto increment

- table origine

In [51]:
#ALTER TABLE test1 ADD COLUMN id SERIAL PRIMARY KEY;

In [52]:
sql = """ALTER TABLE origine ADD COLUMN id SERIAL PRIMARY KEY;"""

In [2]:
import psycopg2
import os

In [54]:
conn = psycopg2.connect(database="collection", user='postgres', password=os.environ.get("pg_psw"), host='localhost', port= '5432')

conn.autocommit = True

cursor = conn.cursor()

sql = """ALTER TABLE origine ADD COLUMN id SERIAL PRIMARY KEY;"""

cursor.execute(sql)
print("Ajout effectué avec succès !")

conn.close()

Ajout effectué avec succès !


- table producteur

In [55]:
conn = psycopg2.connect(database="collection", user='postgres', password=os.environ.get("pg_psw"), host='localhost', port= '5432')

conn.autocommit = True

cursor = conn.cursor()

sql = """ALTER TABLE producteur ADD COLUMN id SERIAL PRIMARY KEY;"""

cursor.execute(sql)
print("Ajout effectué avec succès !")

conn.close()

Ajout effectué avec succès !


- table produit

In [58]:
conn = psycopg2.connect(database="collection", user='postgres', password=os.environ.get("pg_psw"), host='localhost', port= '5432')

conn.autocommit = True

cursor = conn.cursor()

sql = """ALTER TABLE produit ADD COLUMN id SERIAL PRIMARY KEY;"""

cursor.execute(sql)
print("Ajout effectué avec succès !")

conn.close()

Ajout effectué avec succès !


In [3]:
conn = psycopg2.connect(database="collection", user='postgres', password=os.environ.get("pg_psw"), host='localhost', port= '5432')

conn.autocommit = True

cursor = conn.cursor()

sql = '''ALTER TABLE producteur ADD FOREIGN KEY (id) REFERENCES origine(ORIGINE);
    '''

cursor.execute(sql)
print("Relation créée avec respect et robustesse !")

conn.close()

UndefinedColumn: ERREUR:  la colonne « origine » référencée dans la contrainte de clé étrangère n'existe pas


UPDATE niveau
         SET mesure_id = mesure.mesure_id
         FROM mesure
         WHERE mesure.polluant = niveau.polluant;

In [ ]:
conn = psycopg2.connect(database="collection", user='postgres', password=os.environ.get("pg_psw"), host='localhost', port= '5432')

conn.autocommit = True

cursor = conn.cursor()

sql = '''UPDATE producteur SET origine_id = origine.id FROM origine WHERE mesure.polluant = niveau.polluant;ALTER TABLE producteur ADD FOREIGN KEY (id) REFERENCES origine(ORIGINE);
    '''

cursor.execute(sql)
print("Relation créée avec respect et robustesse !")

conn.close()